In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:

spotify = spark.read.format('delta').load('/home/mlops/project/DeltaLake/bronze_data/spotify_user_data_table')
spotify.show()

+--------------------+--------------------+----------+---------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                href|                 uri|popularity|followers|                  id|                name|              genres|  type|                 uid|         create_time|         update_time|           read_time|                 url|           image_url|
+--------------------+--------------------+----------+---------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|https://api.spoti...|spotify:artist:71...|        83| 21527801|711MCceyCBcFnzjGY...|               AC/DC|[australian rock,...|artist|uagVX8DeaePP88Qz5...|2021-12-11 12:21:...|2021-12-11 12:21:...|2022-01-09 16:38:...

In [8]:
import pyspark.sql.functions as F

spotify_cleansed = spotify
spotify_cleansed = spotify_cleansed.filter(F.size("genres") > 0)

spotify_cleansed.show()

+--------------------+--------------------+----------+---------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                href|                 uri|popularity|followers|                  id|                name|              genres|  type|                 uid|         create_time|         update_time|           read_time|                 url|           image_url|
+--------------------+--------------------+----------+---------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|https://api.spoti...|spotify:artist:71...|        83| 21527801|711MCceyCBcFnzjGY...|               AC/DC|[australian rock,...|artist|uagVX8DeaePP88Qz5...|2021-12-11 12:21:...|2021-12-11 12:21:...|2022-01-09 16:38:...

In [9]:
spotify_cleansed.write.format("delta").mode("overwrite").save("/home/mlops/project/DeltaLake/gold_data/spotify_cleansed_table")

In [4]:

spotify = spark.read.format('delta').load('/home/mlops/project/DeltaLake/gold_data/spotify_cleansed_table')
spotify.to_pandas_on_spark().head()


,href,uri,popularity,followers,id,name,genres,type,uid,create_time,update_time,read_time,url,image_url
0,https://api.spotify.com/v1/artists/711MCceyCBc...,spotify:artist:711MCceyCBcFnzjGY4Q7Un,83,21527801,711MCceyCBcFnzjGY4Q7Un,AC/DC,"[australian rock, hard rock, rock]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/711MCceyCBcFnz...,https://i.scdn.co/image/ab6761610000e5ebc4c775...
1,https://api.spotify.com/v1/artists/3JsMj0DEzyW...,spotify:artist:3JsMj0DEzyWc0VDlHuy9Bx,71,2450429,3JsMj0DEzyWc0VDlHuy9Bx,Supertramp,"[album rock, art rock, classic rock, glam rock...",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/3JsMj0DEzyWc0V...,https://i.scdn.co/image/eebed895cad509b17c7c6f...
2,https://api.spotify.com/v1/artists/11YTrgsi6Vj...,spotify:artist:11YTrgsi6VjRCkExQBY13k,50,115537,11YTrgsi6VjRCkExQBY13k,Wise Guys,"[german pop, german rock, kabarett, liedermacher]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/11YTrgsi6VjRCk...,https://i.scdn.co/image/4f1b2cabff7abd06fb17ed...
3,https://api.spotify.com/v1/artists/492hDmhPyuI...,spotify:artist:492hDmhPyuIjP3MgTcIqgm,55,301844,492hDmhPyuIjP3MgTcIqgm,Caro Emerald,"[dutch pop, electro swing, nu jazz]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/492hDmhPyuIjP3...,https://i.scdn.co/image/ab6761610000e5eb379e88...
4,https://api.spotify.com/v1/artists/06bDwgCHeMA...,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,69,708011,06bDwgCHeMAwhgI8il4Y5k,Set It Off,"[electropowerpop, modern rock, otacore, pixie,...",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/06bDwgCHeMAwhg...,https://i.scdn.co/image/ab6761610000e5eba368e7...
